In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import catheat
import openpyxl as xl
from openpyxl.utils.dataframe import dataframe_to_rows
import logging

In [ ]:
# fn =
# fn = T979199.xlsx
# fn = 'T962919_2020-01-23.xlsx'
# fn = 'T1004640_2-14-20.xlsx'
# fn = 'T965----05_2020-02-28.xlsx'
# fn = '20200323_plateT965523.xlsx'

In [26]:
def parseFrame(fn):
    df = pd.read_excel(fn, dtype={
        'Column': str,
        'Tag': str
    }).dropna(how='all')
    if 'Strain' in df.columns:
        df = df.rename(columns={'Strain': 'Lineage'})
    df.Column = df.Column.str.zfill(2)
    df.Lineage = df.Lineage.fillna("--")
    df.Tag = df.Tag.apply(lambda x: "t" + str(x) if x.isnumeric() else x)

    df['Info'] = [
        '\n'.join([str(x), str(y)]) for x, y in zip(df['Tag'], df['Lineage'])
    ]
    locs = pd.read_pickle('96-wellLocs.pkl').sort_values(
        ['Column',
         'Row']).reset_index(drop=True)  # impt standard plate indexing
    df = locs.merge(df, how='outer')
    return df

In [ ]:
# def makeExcel(fn, wb, df):
#     '''Takes the filename for naming the sheet and a pyxl workbook'''
#     sheetname = fn.rstrip('.xlsx') + '_CleanLayout'
#     wb = xl.load_workbook(fn)
#     ws = wb.create_sheet(sheetname)    
#     layout = df.pivot(index='Row', columns='Column',
#                       values=['Info']).fillna("empty")
#     rows = dataframe_to_rows(layout['Info'],
#                              index=True,
#                              header=True)
#     for r in rows:
#         ws.append(r)
#     for cell in ws['A'] + ws[1]:
#         cell.style = 'Pandas'
#     wb.save(fn)
#     return fn

In [3]:
def makeCleanLayout(fn, wb, df):
    sheetname = fn.rstrip('.xlsx') + '_CleanLayout'
    ws = wb.create_sheet(sheetname)    
    layout = df.pivot(index='Row', columns='Column',
                      values=['Info']).fillna("empty")
    rows = dataframe_to_rows(layout['Info'],
                             index=True,
                             header=True)
    for r in rows:
        ws.append(r)
    for cell in ws['A'] + ws[1]:
        cell.style = 'Pandas'
    return wb

In [4]:
def makeCleanSampleList(fn, wb, df):
    sheetname = fn.rstrip('.xlsx') + "_CleanSampleList"
    ws = wb.create_sheet(sheetname)
    cleanSamples = df[['Plate', 'Row', 'Column', 'Tag', 'Lineage']]
    rows = dataframe_to_rows(cleanSamples,
                             index=True,
                             header=True)
    for r in rows:
        ws.append(r)
    return wb

In [5]:
fn = 'T965522_20200421.xlsx'

In [28]:
wb = xl.load_workbook(fn)
df = parseFrame(fn)
# fn = "testoutput.xlsx"

In [29]:
df

,Row,Column,Tag,Unnamed: 3,Unnamed: 4,Unnamed: 5,Sex,Color,Unnamed: 8,Unnamed: 9,DOB,Unnamed: 11,Parents,Unnamed: 13,Lineage,Plate,Info
0,A,01,t5258,NaN,NaN,NaN,M,Black,NaN,NaN,2020-03-24,NaN,2728 x 2713,NaN,iCre,T965522,t5258\niCre
1,B,01,t5259,NaN,1 cage,NaN,F,Black,NaN,NaN,2020-03-16,NaN,2549 x 2663,NaN,Cas 9,T965522,t5259\nCas 9
2,C,01,t5260,NaN,1 cage,NaN,M,Black,NaN,NaN,2020-03-16,NaN,2549 x 2663,NaN,Cas 9,T965522,t5260\nCas 9
3,D,01,t5261,NaN,1 cage,NaN,M,Black,NaN,NaN,2020-03-12,NaN,male x 2535,NaN,iCre,T965522,t5261\niCre
4,E,01,t5262,NaN,NaN,NaN,M,Black,NaN,NaN,2020-03-12,NaN,male x 2535,NaN,iCre,T965522,t5262\niCre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,D,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
92,E,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
93,F,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
94,G,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
wb = makeCleanLayout(fn, wb, df)
wb = makeCleanSampleList(fn, wb, df)

/home/gremlin/.conda/envs/pipelinePrep/lib/python3.7/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [31]:
wb.save(fn)

In [ ]:
fns = ['T979159_2020-01-27.xlsx', 'T979199.xlsx', 'T962919_2020-01-23.xlsx', 'T1004640_2-14-20.xlsx']

In [ ]:
for fn in fns:
    wb = xl.load_workbook(fn)
    df = parseFrame(fn)
    wb = makeCleanLayout(fn, wb, df)
    wb = makeCleanSampleList(fn, wb, df)
    wb.save(fn)